In [ ]:
from dolfin import *
import numpy as np
import os
import json

In [ ]:
# Define input and output paths
MESH_FILE = "mesh/output/mesh.xdmf"
SURFACE_MARKERS_FILE = "mesh/output/surface.xdmf"
SOLID_MARKERS_FILE = "mesh/output/solid.xdmf"
MARKERS_JSON_FILE = "mesh/output/markers.json"
RESULTS_OUTPUT_FILE = "post-processing/output/cantilever_results.xdmf"

In [ ]:
# ===== USER INPUT PARAMETERS =====

# Material Properties
# Material 1 (M1) - Steel
E1 = 2.1e11  # Young's modulus (Pa)
nu1 = 0.3    # Poisson's ratio

# Material 2 (M2) - Aluminum
E2 = 0.7e11  # Young's modulus (Pa)  
nu2 = 0.33   # Poisson's ratio

# Loading Parameters
f_body_x = 0.0     # Body force in x-direction (N/m³)
f_body_y = 0.0     # Body force in y-direction (N/m³)
f_body_z = 0.0     # Body force in z-direction (N/m³)

f_surface_x = 0.0    # Surface traction in x-direction (Pa)
f_surface_y = 0.0    # Surface traction in y-direction (Pa)
f_surface_z = -1e6   # Surface traction in z-direction (Pa)

print("=== USER INPUT PARAMETERS ===")
print(f"Material 1 (Steel): E = {E1:.2e} Pa, nu = {nu1}")
print(f"Material 2 (Aluminum): E = {E2:.2e} Pa, nu = {nu2}")
print(f"Body force: ({f_body_x}, {f_body_y}, {f_body_z}) N/m³")
print(f"Surface traction: ({f_surface_x}, {f_surface_y}, {f_surface_z}) Pa")
print("==============================")

In [ ]:
# Load markers information from JSON
with open(MARKERS_JSON_FILE, 'r') as f:
    markers_info = json.load(f)

# Load mesh
mesh = Mesh()
with XDMFFile(MESH_FILE) as infile:
    infile.read(mesh)

# Load surface markers (for boundary conditions and loads)
mvc_surface = MeshValueCollection("int", mesh, 2)
with XDMFFile(SURFACE_MARKERS_FILE) as infile:
    infile.read(mvc_surface, "marker")

surface_markers = cpp.mesh.MeshFunctionInt(mesh, mvc_surface)

# Load solid markers (for material properties)
mvc_solid = MeshValueCollection("int", mesh, 3)
with XDMFFile(SOLID_MARKERS_FILE) as infile:
    infile.read(mvc_solid, "marker")

solid_markers = cpp.mesh.MeshFunctionInt(mesh, mvc_solid)

# Print loading summary
print("=== MESH AND MARKERS LOADED ===")
print(f"Mesh: {mesh.num_vertices()} vertices, {mesh.num_cells()} cells")
print(f"Surface markers: {markers_info['surface']}")
print(f"Solid markers: {markers_info['solid']}")
print("=================================")

In [ ]:
# ===== SETUP MARKERS AND FUNCTION SPACE =====
FIXED_MARKER = markers_info['surface']['Fixed']['id']
FORCED_MARKER = markers_info['surface']['Forced']['id']
M1_MARKER = markers_info['solid']['M1']['id']
M2_MARKER = markers_info['solid']['M2']['id']

V = VectorFunctionSpace(mesh, 'P', 1)

# ===== MATERIAL PROPERTIES DEFINITION =====
material_properties = {
    M1_MARKER: {'E': E1, 'nu': nu1, 'name': 'Steel'},
    M2_MARKER: {'E': E2, 'nu': nu2, 'name': 'Aluminum'}
}

# ===== ASSIGN MATERIAL PROPERTIES TO MESH CELLS =====
mu_values = np.ones(mesh.num_cells()) * E1 / (2 * (1 + nu1))  # Default to material 1
lambda_values = np.ones(mesh.num_cells()) * E1 * nu1 / ((1 + nu1) * (1 - 2 * nu1))  # Default to material 1

for c in cells(mesh):
    marker_id = int(solid_markers[c])
    if marker_id in material_properties:
        E = material_properties[marker_id]['E']
        nu = material_properties[marker_id]['nu']
        mu_val = E / (2 * (1 + nu))
        lambda_val = E * nu / ((1 + nu) * (1 - 2 * nu))
        mu_values[c.index()] = mu_val
        lambda_values[c.index()] = lambda_val

mu_func = Function(FunctionSpace(mesh, "DG", 0))
mu_func.vector().set_local(mu_values)
mu_func.vector().apply("insert")

lambda_func = Function(FunctionSpace(mesh, "DG", 0))  
lambda_func.vector().set_local(lambda_values)
lambda_func.vector().apply("insert")

# ===== BOUNDARY CONDITIONS SETUP =====
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1, 0)

for facet in facets(mesh):
    facet_index = facet.index()
    if facet_index < len(surface_markers.array()):
        marker_val = surface_markers.array()[facet_index]
        if marker_val == FIXED_MARKER:
            boundaries[facet] = 1  # Mark fixed boundary
        elif marker_val == FORCED_MARKER:
            boundaries[facet] = 2  # Mark forced boundary

bc = DirichletBC(V, Constant((0, 0, 0)), boundaries, 1)

# ===== PHYSICS: ELASTICITY FORMULATION =====
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

def sigma(u):
    d = u.geometric_dimension()
    return lambda_func*div(u)*Identity(d) + 2*mu_func*epsilon(u)

u = TrialFunction(V)
v = TestFunction(V)

a = inner(sigma(u), epsilon(v))*dx

# ===== LOADING CONDITIONS =====
f_body = Constant((f_body_x, f_body_y, f_body_z))  # Body force
f_surface = Constant((f_surface_x, f_surface_y, f_surface_z))  # Surface traction on forced surface

ds = Measure('ds', domain=mesh, subdomain_data=boundaries)
L = dot(f_body, v)*dx + dot(f_surface, v)*ds(2)  # Apply load on marker 2 (forced surface)

# ===== SOLVE THE SYSTEM =====
u_sol = Function(V, name="Displacement")
solve(a == L, u_sol, bc)

# ===== POST-PROCESSING: STRESS CALCULATIONS =====
def von_mises_stress(u):
    s = sigma(u) - (1./3)*tr(sigma(u))*Identity(3)  # deviatoric stress
    von_mises = sqrt(3./2*inner(s, s))
    return von_mises

V_scalar = FunctionSpace(mesh, 'P', 1)
von_mises_expr = von_mises_stress(u_sol)
von_mises_sol = project(von_mises_expr, V_scalar)
von_mises_sol.rename("VonMisesStress", "")

mu_projected = project(mu_func, V_scalar)
mu_projected.rename("ShearModulus", "")
lambda_projected = project(lambda_func, V_scalar)
lambda_projected.rename("LameParameter", "")

# ===== EXPORT RESULTS =====
with XDMFFile(RESULTS_OUTPUT_FILE) as xdmf:
    xdmf.parameters["flush_output"] = True
    xdmf.parameters["functions_share_mesh"] = True
    xdmf.write(u_sol, 0.0)
    xdmf.write(von_mises_sol, 0.0)
    xdmf.write(mu_projected, 0.0)
    xdmf.write(lambda_projected, 0.0)

# ===== ANALYSIS AND RESULTS SUMMARY =====
V_DG = FunctionSpace(mesh, "DG", 0)
von_mises_projected = project(von_mises_expr, V_DG)
von_mises_cell_values = von_mises_projected.vector().get_local()

print(f"\n=== ANALYSIS RESULTS ===")
print(f"Markers used: Fixed={FIXED_MARKER}, Forced={FORCED_MARKER}, M1={M1_MARKER}, M2={M2_MARKER}")
print(f"Material properties:")
for marker_id, props in material_properties.items():
    print(f"  {props['name']} (marker {marker_id}): E={props['E']:.2e} Pa, nu={props['nu']}")

print(f"\nSolution statistics:")
print(f"  Maximum displacement: {u_sol.vector().max():.6e} m")
print(f"  Maximum von Mises stress: {von_mises_sol.vector().max():.6e} Pa")
print(f"  Results saved to: {RESULTS_OUTPUT_FILE}")

print("=== ANALYSIS COMPLETE ===")